<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Multimodal_Transportation_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Multimodal Transportation Problem**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data

!pip install scgraph==2.1.0         # Python package used to compute paths and distances on a real-world transportation network
!pip install scgraph_data==2.0.0    # Python package used to compute paths and distances on a real-world transportation network



In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


import scgraph                                                  # For computing paths and distances on a real-world transportation network
from scgraph.geographs.us_freeway import us_freeway_geograph    # Data on US highway network (for road paths and distances)
from scgraph.geographs.marnet import marnet_geograph            # Data on maritime routes (for ocean paths and distances)


## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               color,                       # Color of the marker icon
               background_color,            # Background color of the marker icon
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=color,
                border_color=color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing the shortest path between two points on a real road or ocean network**

In [ ]:
# Function for computing the shortest path between two points on a real road or ocean network

def shortest_path(origin, destination, mode, result, unit='mi'):

    # Extract coordinates from origin and destination objects
    origin_coordinates = (origin.lat, origin.lon)
    destination_coordinates = (destination.lat, destination.lon)

    # Calculate the shortest path on the ocean network
    if mode == 'ocean':
        output = marnet_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Calculate the shortest path on the road network
    elif mode == 'road':
        output = us_freeway_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Return the total distance of the path
    if result == 'distance':
        return output['length']

    # Return the coordinates representing the path
    elif result == 'coordinate_path':
        return output['coordinate_path']


### **Plotting flows on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows(map,                   # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color=color,                                # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map


### **Ploting flows (real network) on the map**

In [ ]:
# Functions that adjust the arc path to ensure longitude continuity across the globe

def adjustArcPath(path):
    for index in range(1, len(path)):
        x = path[index][1]
        prevX = path[index - 1][1]
        path[index][1] = x - (round((x - prevX)/360,0) * 360)
    return path

def modifyArcPathLong(points, amount):
    return [[i[0], i[1]+amount] for i in points]

def getCleanArcPath(path):
    path = adjustArcPath(path)
    return [
        path,
        modifyArcPathLong(path, 360),
        modifyArcPathLong(path, -360),
        modifyArcPathLong(path, 720),
        modifyArcPathLong(path, -720)
    ]

# Plots real flows on a Folium map based on optimization model results

def plot_real_flows(map,              # Folium map object where flows will be plotted.
               vars,                  # Dictionary of flow decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               mode,                  # Transportation mode (road or ocean)
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the shortest path coordinates for the node pair
            path = shortest_path(nodes[node1_key], nodes[node2_key], mode = mode, result = 'coordinate_path')

            # Add the path as a polyline on the map, with width proportional to the flow value
            folium.PolyLine(getCleanArcPath(path),
                      color=color,
                      weight=var.X / max_val * max_width,   # Normalize weight by maximum flow value
                      opacity=opacity).add_to(map)


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/multimodal_transportation/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon,country,demand
0,c1,Atlanta,33.7628,-84.4220,United States,270
1,c2,Portland,45.5371,-122.6500,United States,120
2,c3,Providence,41.8230,-71.4187,United States,90
3,c4,Indianapolis,39.7771,-86.1458,United States,90
4,c5,Pittsburgh,40.4397,-79.9763,United States,120


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/multimodal_transportation/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

,ID,name,country,lat,lon,supply
0,COBUN,Buenaventura,Colombia,3.889934,-77.078605,620
1,LKCMB,Colombo,Sri Lanka,6.886693,79.918738,720
2,INJHT,Jawaharlal Nehru,India,18.502265,73.855672,480
3,PECAL,Callao,Peru,-12.052263,-77.139113,440
4,CLSAI,San Antonio,Chile,-33.580861,-71.613238,240


In [ ]:
# File containing port data
port_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/multimodal_transportation/ports.csv'

# Loading port data into a pandas DataFrame
ports_df = pd.read_csv(port_data_file)

# Displaying the first few rows of the DataFrame to verify the data
ports_df.head()


,ID,name,country,lat,lon
0,USBAL,Baltimore,United States,39.290882,-76.610759
1,USBOS,Boston,United States,42.355433,-71.060511
2,USCHS,Charleston,United States,32.787601,-79.940273
3,USHOU,Houston,United States,29.758938,-95.367697
4,USJAX,Jacksonville,United States,30.332184,-81.655651


#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude
        self.demand = demand      # Customer's demand


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply


In [ ]:
# Class representing a Port object

class Port():
    def __init__(self, ID, name, lat, lon):
        self.ID = ID            # Port's ID
        self.name = name        # Ports's name
        self.lat = lat          # Ports's latitude
        self.lon = lon          # Ports's longitude

#### Creating node objects

In [ ]:
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'])   # Customer's demand

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**customers, **suppliers}

In [ ]:
# Creating a dictionary of port objects
ports = dict()
for i, row in ports_df.iterrows():
    ports[row['ID']] = Port(ID = row['ID'],                       # Port's ID
                                    name = row['name'],           # Port's name
                                    lat = row['lat'],             # Port's latitude
                                    lon = row['lon'])             # Port's longitude

# Merging the port dictionary into the existing nodes dictionary
nodes = {**nodes, **ports}

#### Visualizing node objects

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=2)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', color='green', background_color='yellow')

# Plot port locations with an anchor icon, blue color, and white background
plot_nodes(map=map, nodes=ports, icon='anchor', color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary to store distances between suppliers and ports, and ports and customers

distances = dict()  # Initialize an empty dictionary to store distances

# Calculate distances between each supplier and port using the ocean network
for s, supplier in suppliers.items():
    for p, port in ports.items():
        # Store the distance between each supplier-port pair in the dictionary
        distances[s, p] = shortest_path(origin=supplier, destination=port, mode='ocean', result='distance', unit='km')

# Calculate distances between each port and customer using the road network
for p, port in ports.items():
    for c, customer in customers.items():
        # Store the distance between each port-customer pair in the dictionary
        distances[p, c] = shortest_path(origin=port, destination=customer, mode='road', result='distance', unit='km')


#### Arc costs

In [ ]:
# Creating a dictionary to store unit transportation costs between different nodes
unit_cost = dict()


# Unit transportation costs beteween suppliers and ports (ocean)

# Cost per kilometer for ocean transport
cost_km_ocean = 0.2

# Calculate the ocean transport cost for each supplier-port pair
for s, supplier in suppliers.items():
    for p, port in ports.items():
        # Ocean transport cost is calculated by multiplying distance by cost per kilometer
        unit_cost[s, p] = distances[s, p] * cost_km_ocean


# Unit transportation costs beteween ports and customers (road)

# Minimum transportation cost for short distances
min_cost = 1000

# Distance threshold to differentiate between long and short distance rates
distance_threshold = 1500

# Cost per kilometer for long-distance road transport
long_distance_rate = 3.5

# Cost per kilometer for short-distance road transport
short_distance_rate = 5

# Calculate the road transport cost for each port-customer pair
for p, port in ports.items():
    for c, customer in customers.items():
        # Apply long-distance rate if the distance exceeds the threshold
        if distances[p, c] > distance_threshold:
            unit_cost[p, c] = distances[p, c] * long_distance_rate
        # Apply short-distance rate, ensuring cost does not fall below minimum cost
        else:
            unit_cost[p, c] = max(min_cost, distances[p, c] * short_distance_rate)


## **Optimization model**

### **Creating and solving the optimization model**

In [ ]:
# Initializing the model
model = grb.Model("Multimodal Transportation Problem")

# Creating decision variables
flow_vars_ocean = dict()
flow_vars_road = dict()

for s in suppliers:
  for p, port in ports.items():
        flow_vars_ocean[s, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_ocean_{0}_{1}".format(s, p))

for p, port in ports.items():
  for c in customers:
        flow_vars_road[p, c] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_road_{0}_{1}".format(p, c))


# Creating the objective function

cost_supplier_port = grb.quicksum(unit_cost[s, p] * flow_vars_ocean[s, p] for s in suppliers for p in ports)
cost_port_customer = grb.quicksum(unit_cost[p, c] * flow_vars_road[p, c] for p in ports for c in customers)

total_cost = cost_supplier_port + cost_port_customer

model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding demand constraints
for c, customer in customers.items():
    model.addConstr(grb.quicksum(flow_vars_road[p, c] for p in ports) == customer.demand, c)

# Adding supply constraints
for s, supplier in suppliers.items():
    model.addConstr(grb.quicksum(flow_vars_ocean[s, p] for p in ports) <= supplier.supply, s)

# Adding flow preservation constraints
for p in ports:
    model.addConstr(grb.quicksum(flow_vars_ocean[s, p] for s in suppliers) == grb.quicksum(flow_vars_road[p, c] for c in customers), p)

# Solving
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 32 rows, 240 columns and 480 nonzeros
Model fingerprint: 0x8edb8ef6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 7e+02]
Presolve time: 0.01s
Presolved: 32 rows, 240 columns, 480 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5503735e+06   3.100000e+03   0.000000e+00      0s
      36    1.5784509e+07   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.578450906e+07


## **Solution visualization and analysis**

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=2)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', color='green', background_color='yellow')

# Plot port locations with an anchor icon, blue color, and white background
plot_nodes(map=map, nodes=ports, icon='anchor', color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Plot the flows between suppliers and ports with a maximum line width of 20
plot_flows(map=map, max_width=20, vars=flow_vars_ocean, nodes = nodes, color = 'blue')

# Plot the flows between ports and customers with a maximum line width of 20
plot_flows(map=map, max_width=20, vars=flow_vars_road, nodes = nodes, color = 'orange')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map



In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=2)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', color='green', background_color='yellow')

# Plot port locations with an anchor icon, blue color, and white background
plot_nodes(map=map, nodes=ports, icon='anchor', color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Plot the flows between suppliers and ports with a maximum line width of 20
plot_real_flows(map=map, max_width=20, vars=flow_vars_ocean, nodes = nodes, mode = 'ocean', color = 'blue')

# Plot the flows between ports and customers with a maximum line width of 20
plot_real_flows(map=map, max_width=20, vars=flow_vars_road, nodes = nodes, mode = 'road', color = 'orange')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map

